In [ ]:
import pandas as pd
import sqlite3
import random
from contextlib import contextmanager

# Step 1: Generate sample weather data (similar to previous request)
def generate_sample_data(num_records=1000):
    years = list(range(1900, 2021))
    data = {
        'record_id': range(1, num_records + 1),
        'year': [random.choice(years) for _ in range(num_records)],
        'temperature_c': [random.uniform(-50, 50) for _ in range(num_records)]
    }
    return pd.DataFrame(data)

# Step 2: Simulate Hive database and table in SQLite
@contextmanager
def sqlite_connection(db_name):
    conn = sqlite3.connect(db_name)
    try:
        yield conn
    finally:
        conn.close()

def setup_hive_like_db():
    # Create a SQLite database (simulating Hive database)
    db_name = 'weather_hive.db'
    df = generate_sample_data(1000)

    with sqlite_connection(db_name) as conn:
        # Create table (simulating Hive CREATE TABLE)
        df.to_sql('weather_data', conn, if_exists='replace', index=False)

        # Create index on year (simulating Hive CREATE INDEX)
        conn.execute('CREATE INDEX idx_year ON weather_data(year)')

        # Create view (simulating Hive CREATE VIEW)
        conn.execute('''
            CREATE VIEW positive_temps AS
            SELECT record_id, year, temperature_c
            FROM weather_data
            WHERE temperature_c > 0
        ''')

    print(f"Database '{db_name}', table 'weather_data', index 'idx_year', and view 'positive_temps' created.")

# Step 3: User-Defined Function (UDF) to convert Celsius to Fahrenheit
def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9/5) + 32

# Register UDF in SQLite
def register_udf(conn):
    conn.create_function('c_to_f', 1, celsius_to_fahrenheit)
    print("UDF 'c_to_f' registered.")

# Step 4: Generate weather report (min/max temps per year)
def generate_weather_report():
    db_name = 'weather_hive.db'
    with sqlite_connection(db_name) as conn:
        # Register UDF
        register_udf(conn)

        # Query table for min/max temps
        query_table = '''
            SELECT year,
                   MIN(temperature_c) AS min_temp_c,
                   MAX(temperature_c) AS max_temp_c
            FROM weather_data
            GROUP BY year
            ORDER BY year
        '''
        report_df = pd.read_sql_query(query_table, conn)

        # Query view with UDF for max temp in Fahrenheit
        query_view = '''
            SELECT year,
                   c_to_f(MAX(temperature_c)) AS max_temp_f
            FROM positive_temps
            GROUP BY year
            ORDER BY year
        '''
        view_df = pd.read_sql_query(query_view, conn)

        # Merge results
        result = report_df.merge(view_df, on='year', how='left')
        result['max_temp_f'] = result['max_temp_f'].round(1)
        result['min_temp_c'] = result['min_temp_c'].round(1)
        result['max_temp_c'] = result['max_temp_c'].round(1)

    return result

# Step 5: Run the POC
if __name__ == "__main__":
    print("Setting up Hive-like environment...")
    setup_hive_like_db()

    print("\nGenerating Weather Temperature Statistics Report...")
    report = generate_weather_report()

    print("\n=== Weather Report ===")
    print("Year\tMin Temp (°C)\tMax Temp (°C)\tMax Temp (°F)")
    print("-" * 50)
    for _, row in report.iterrows():
        print(f"{int(row['year'])}\t{row['min_temp_c']}\t\t{row['max_temp_c']}\t\t{row['max_temp_f']}")

    print("\nSample data from view (first 5 rows):")
    with sqlite_connection('weather_hive.db') as conn:
        sample_view = pd.read_sql_query('SELECT * FROM positive_temps LIMIT 5', conn)
        print(sample_view)

Setting up Hive-like environment...
Database 'weather_hive.db', table 'weather_data', index 'idx_year', and view 'positive_temps' created.

Generating Weather Temperature Statistics Report...
UDF 'c_to_f' registered.

=== Weather Report ===
Year	Min Temp (°C)	Max Temp (°C)	Max Temp (°F)
--------------------------------------------------
1900	-45.9		28.2		82.7
1901	-46.1		28.8		83.8
1902	-43.5		22.5		72.5
1903	-38.6		47.4		117.3
1904	-43.4		43.6		110.5
1905	-48.3		46.5		115.6
1906	-19.7		49.2		120.6
1907	6.3		30.4		86.8
1908	-48.5		34.8		94.6
1909	-20.9		24.5		76.1
1910	-48.4		46.3		115.4
1911	-42.4		30.6		87.1
1912	-42.2		48.1		118.5
1913	-45.5		41.9		107.4
1914	-46.7		22.2		72.0
1915	-49.8		44.7		112.5
1916	-49.8		43.9		111.0
1917	-7.1		35.1		95.3
1918	-18.4		38.5		101.3
1919	-35.8		39.3		102.7
1920	-45.4		42.9		109.3
1921	-35.9		46.0		114.8
1922	-43.7		44.0		111.1
1923	-40.1		2.9		37.1
1924	14.9		38.2		100.7
1925	-42.5		37.3		99.2
1926	-48.2		49.7		121.5
1927	-48.7		48.8		119.9
1928	